In [7]:
import matplotlib.pyplot as plt
import os
import numpy as np
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings("ignore")

!pip install metric-learn
from sklearn.neighbors import KNeighborsClassifier
from metric_learn import MLKR

In [2]:
import scipy.io
data = scipy.io.loadmat('PaviaU.mat')['paviaU']
gt = scipy.io.loadmat('PaviaU_gt.mat')['paviaU_gt']

In [3]:
def applyPCA(X, numComponents=15):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0],X.shape[1], numComponents))
    return newX

In [4]:
data=applyPCA(data)

df = pd.DataFrame(data.reshape(data.shape[0]*data.shape[1], -1))
df['class'] = gt.ravel()

X = df[df['class']!=0].iloc[:, :-1].values

y = df[df['class']!=0].iloc[:, -1].values

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=42)

mlkr = MLKR(verbose=True, max_iter=100)
knn = KNeighborsClassifier(n_neighbors=5)

# MLKR modelini eğitim verilerine uygula
mlkr.fit(X_train, y_train)

# KNN modelini MLKR dönüştürülmüş eğitim verilerine uygula
knn.fit(mlkr.transform(X_train), y_train)

# Modeli test verileri üzerinde değerlendir
mlkr_acc = knn.score(mlkr.transform(X_test), y_test)
print('MLKR accuracy:', mlkr_acc)

[MLKR]
[MLKR]  Iteration      Objective Value    Time(s)
[MLKR] ------------------------------------------
[MLKR]          0         5.126130e+04      18.67
[MLKR]          1         3.437417e+04      18.26
[MLKR]          2         2.892506e+04      18.21
[MLKR]          3         2.238352e+04      18.21
[MLKR]          4         1.935797e+04      19.16
[MLKR]          5         1.863113e+04      19.73
[MLKR]          6         1.694843e+04      20.01
[MLKR]          7         1.638278e+04      20.34
[MLKR]          8         1.611253e+04      20.42
[MLKR]          9         1.588566e+04      20.54
[MLKR]         10         1.560914e+04      20.64
[MLKR]         11         1.539037e+04      20.71
[MLKR]         12         1.526551e+04      20.80
[MLKR]         13         1.511685e+04      20.75
[MLKR]         14         1.493538e+04      21.05
[MLKR]         15         1.482960e+04      21.04
[MLKR]         16         1.469985e+04      21.25
[MLKR]         17         1.461401e+04     

In [9]:
import joblib

joblib.dump(mlkr, 's_mlkr_pu_model.pkl')
joblib.dump(knn,'s_mlkr_pu_knn_model.pkl')

mlkr_loaded = joblib.load('s_mlkr_pu_model.pkl')
knn_loaded =joblib.load('s_mlkr_pu_knn_model.pkl')

lfda_data=mlkr_loaded.transform(X)
pred=knn_loaded.predict(lfda_data)

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

print("accuracy:",accuracy_score(pred,y))
print("cohen kappa:", cohen_kappa_score(pred,y))
print("f1 score:",f1_score(pred,y,average='macro'))

accuracy: 0.9613802132036656
cohen kappa: 0.948604578147515
f1 score: 0.9531182239581236
